In [7]:
import os
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("data_exploration").getOrCreate()

In [8]:
PARENT_DATA_PATH = "starter/"
customer_landing_df = spark.read.json(PARENT_DATA_PATH + "customer/landing/")
accelerometer_landing_df = spark.read.json(PARENT_DATA_PATH + "accelerometer/landing")
step_trainer_landing_df = spark.read.json(PARENT_DATA_PATH + "step_trainer/landing")

In [9]:
print(len(accelerometer_landing_df.columns))
print(accelerometer_landing_df.columns)


5
['timestamp', 'user', 'x', 'y', 'z']


In [10]:

customer_data_count = spark.read.json(PARENT_DATA_PATH + "customer/landing/").count()
assert customer_data_count == 956

In [11]:
accelerometer_data_count = spark.read.json(PARENT_DATA_PATH + "accelerometer/landing").count()
assert accelerometer_data_count == 81273


In [12]:
step_trainer_count = spark.read.json(PARENT_DATA_PATH + "step_trainer/landing").count()
# print(step_trainer_count)
assert step_trainer_count == 28680

### define customer curated

In [18]:
customer_trusted_df = customer_landing_df.dropna(subset=["shareWithResearchAsOfDate"])
customer_trusted_df.write.json(PARENT_DATA_PATH + "customer/trusted/", 'overwrite')
assert customer_trusted_df.count() == 482

In [23]:
# customer_trusted_df = spark.read.json(PARENT_DATA_PATH + 'customer/trusted/')
# customer_trusted_df.count()
customer_landing_df.printSchema()

root
 |-- birthDay: string (nullable = true)
 |-- customerName: string (nullable = true)
 |-- email: string (nullable = true)
 |-- lastUpdateDate: long (nullable = true)
 |-- phone: string (nullable = true)
 |-- registrationDate: long (nullable = true)
 |-- serialNumber: string (nullable = true)
 |-- shareWithFriendsAsOfDate: long (nullable = true)
 |-- shareWithPublicAsOfDate: long (nullable = true)
 |-- shareWithResearchAsOfDate: long (nullable = true)



In [24]:
accelerometer_trusted_cols = ['user','timestamp', 'x', 'y', 'z']

customer_accelerometer_trusted_df = customer_trusted_df.withColumnRenamed("email", "user")\
    .join(accelerometer_landing_df, on="user", how="left")


accelerometer_trusted_df = customer_accelerometer_trusted_df.select(*accelerometer_trusted_cols)

In [25]:
accelerometer_trusted_df.count()

40981

In [26]:
# accelerometer_trusted_df.write.json(PARENT_DATA_PATH + "accelerometer/trusted/", 'overwrite')

In [28]:
condition = customer_trusted_df.email == accelerometer_trusted_df.user 

In [36]:
accelerometer_trusted_unique_users_df = accelerometer_trusted_df.drop_duplicates(subset=["user"])
customer_curated_df = customer_trusted_df.join(accelerometer_trusted_unique_users_df, on=condition, how="left")\
.drop("user", "x","y","z","timestamp")

In [40]:
# customer_curated_df.write.json(PARENT_DATA_PATH + "customer/curated/", 'overwrite')


482

In [41]:
step_trainer_cols = ["distanceFromObject","sensorReadingTime", "serialNumber"]
step_trainer_trusted_df = customer_curated_df\
    .join(step_trainer_landing_df, on="serialNumber", how="inner")\
    .select(*step_trainer_cols)
assert step_trainer_trusted_df.count() == 14460

In [44]:
step_trainer_trusted_df.write.json(PARENT_DATA_PATH + "step_trainer/trusted/", 'overwrite')

In [27]:
assert accelerometer_trusted_df.count()==40981

In [46]:
accelerometer_trusted_df.columns

['user', 'timestamp', 'x', 'y', 'z']

In [48]:
condition_step_trainer_accelerometer = accelerometer_trusted_df.timestamp == step_trainer_trusted_df.sensorReadingTime
machine_learning_curated=step_trainer_trusted_df.join(accelerometer_trusted_df, on=condition_step_trainer_accelerometer, how="inner")
machine_learning_curated.show()

+------------------+-----------------+--------------------+--------------------+-------------+----+----+----+
|distanceFromObject|sensorReadingTime|        serialNumber|                user|    timestamp|   x|   y|   z|
+------------------+-----------------+--------------------+--------------------+-------------+----+----+----+
|               282|    1655564471084|c007feb7-dc9b-423...|Danny.Phillips@te...|1655564471084|-1.0| 0.0|-1.0|
|               293|    1655564418085|f6963c38-0052-43a...|Suresh.Spencer@te...|1655564418085| 1.0| 0.0|-1.0|
|               236|    1655564421347|72ec6b97-0bde-484...|Bobby.Jefferson@t...|1655564421347| 0.0| 1.0|-1.0|
|               236|    1655564421347|72ec6b97-0bde-484...|Bobby.Jefferson@t...|1655564421347|-1.0| 0.0| 0.0|
|               253|    1655564145175|9cebd63f-aa88-49b...|David.Khatib@test...|1655564145175| 1.0| 0.0|-1.0|
|               259|    1655564473699|44a10dbd-444c-427...|Jacob.Anderson@te...|1655564473699| 0.0| 1.0| 0.0|
|         

In [ ]:
machine_learning_curated=0